In [32]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA
from ipywidgets import *
from manim import *

np.set_printoptions(precision=3, suppress=True, linewidth=100)
def plotLayout(ax,xlim,ylim,title,xlab,ylab):
    # prepares a plot layout
    # Where:
    #- axis ax: figures axis
    #- param xlim: (xmin,xmax)x range of plot
    #- param ylim: (ymin,ymax)y range of plot

    #   Refine plot
    ax.grid(True)
    ax.axhline(y=0, color='k')
    ax.axvline(x=0, color='k')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_title(title)
    ax.set_xlabel(xlab)
    ax.set_ylabel(ylab)

In [2]:
#import needed: numpy, pyplot, plotLayout
def nd_orbit(fnct, x0, it):
    # calculates orbits of n dimensional map
    # Where:
    #- param fnct: Mapping function mapping x0 to x1 taking a single parameter
    #- param x0: Initial value
    #- param it: Number of iterations
    #- returns: The entire orbit, including the initial value
    #   Initialize
    orbit = np.zeros((len(x0),it))
    orbit[:,0]=x0
    #   Main loop
    for i in range(it-1):
        orbit[:,i+1] = fnct(orbit[:,i])
    return orbit

In [3]:
def activation(x):
    #the actication  function f()
    #return (x+abs(x))/2 #ReLu
    return 1/(1+np.exp(-x)) #sigmoid 
    #return np.tanh(x) #tanh

In [4]:
#import needed: numpy, pyplot, plotLayout, nd_orbit
def neuron_2_discrete(x0, weightM,bias ,Nit=100, ntrans=10):
    # calculates stable solution henon map
    # Where:
    #- param x0: Initial value
    #- param a: Parameters of Henon map, being [a,b] in the standard expression
    #- returns: An array containing the orbit of the Henon map without transients

    linearmap = lambda x: (weightM @ activation(x) + bias)

    #   Calculate orbit
    orbit = nd_orbit(linearmap, x0, Nit)
    orbit = orbit[:, ntrans+1:] #remove the transient (+1 to also remove initial condition)
    return orbit


In [34]:
def orbit_generate():
    return [[-1,-1],[1,-2]]

In [36]:
%%manim -ql -v WARNING TwoNeuronRNNOrbit

class TwoNeuronRNNOrbit(Scene):
    def animate(self,orbit_points):
        # Create axes for the orbit plot
        axes = Axes(
            x_range=[-2, 2, 0.5], y_range=[-2, 2, 0.5],
            x_length=6, y_length=6,
            axis_config={"color": BLUE}
        )
        labels = axes.get_axis_labels(x_label="h1", y_label="h2")
        numberplane = NumberPlane()
        self.add(numberplane,axes, labels)
        #self.play(Create(axes), Write(labels))
        # Convert the simulated points to Manim points on the 2D plane
        orbit_points_manim = [
            axes.c2p(h[0], h[1]) for h in orbit_points
        ]
        # Create the orbit as a line with a moving dot
        orbit_path = VMobject(color=BLUE)
        orbit_path.set_points_as_corners(orbit_points_manim)
        moving_dot = Dot(orbit_points_manim[0], color=RED)

        # Animation: Draw the orbit path and move the dot along the path
        self.play(Create(orbit_path), FadeIn(moving_dot))
        self.play(MoveAlongPath(moving_dot, orbit_path), run_time=8, rate_func=linear)

        # Hold the final state for a moment
        self.wait(2)
    def construct(self):
        orbit_points = orbit_generate()
        self.animate(orbit_points)


Manim Community v0.18.1